In [1]:
import pandas as pd
import numpy as np
import sys
import sklearn
import matplotlib.pyplot as plt
import category_encoders.target_encoder as te
from sklearn.model_selection import StratifiedKFold, KFold
import warnings
warnings.filterwarnings('ignore')



sys.path.append('../')
from Functions import prepro
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

%matplotlib inline

In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv")
test_df = pd.read_csv("../../data/Processed/test2.csv")

train_size = train_df.shape[0]
test_size = test_df.shape[0]
all_size = train_size + test_size

print("train data size : ", train_size)
print("test data size : ", test_size)
print("the ratio of train and test  ", train_size/all_size, " : ", test_size/all_size )
train_df.head()

train data size :  66125
test data size :  28340
the ratio of train and test   0.6999947070343513  :  0.30000529296564865


,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,A2-level,A3-weapon,A3-rank,A3-level,A4-weapon,A4-rank,A4-level,B1-weapon,B1-rank,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y,category1-A1,category2-A1,subweapon-A1,special-A1,mainweapon-A1,reskin-A1,range-main-A1,atack-main-A1,rapid-main-A1,vote-main-A1,fav-main-A1,DPS-main-A1,kill_time_human-main-A1,kill_time_ika-main-A1,front_gap_human-main-A1,front_gap_ika-main-A1,rensya_frame-main-A1,saidai_damege-main-A1,range-bullet-main-A1,range-draw-main-A1,damage_min-sub-A1,damage_max-sub-A1,ink-sub-A1,install_num-sub-A1,good-sub-A1,direct_range-sub-A1,distant-range_sub-A1,damage_min-special-A1,damage_max-special-A1,duration-special-A1,good-special-A1,direct_rad-special-A1,close_rad-special-A1,distant_rad-special-A1,category1-A2,category2-A2,subweapon-A2,special-A2,mainweapon-A2,reskin-A2,range-main-A2,atack-main-A2,rapid-main-A2,vote-main-A2,fav-main-A2,DPS-main-A2,kill_time_human-main-A2,kill_time_ika-main-A2,front_gap_human-main-A2,front_gap_ika-main-A2,rensya_frame-main-A2,saidai_damege-main-A2,range-bullet-main-A2,range-draw-main-A2,damage_min-sub-A2,damage_max-sub-A2,ink-sub-A2,install_num-sub-A2,good-sub-A2,direct_range-sub-A2,distant-range_sub-A2,damage_min-special-A2,damage_max-special-A2,duration-special-A2,good-special-A2,direct_rad-special-A2,close_rad-special-A2,distant_rad-special-A2,category1-A3,category2-A3,subweapon-A3,special-A3,mainweapon-A3,reskin-A3,range-main-A3,atack-main-A3,rapid-main-A3,vote-main-A3,fav-main-A3,DPS-main-A3,kill_time_human-main-A3,kill_time_ika-main-A3,front_gap_human-main-A3,front_gap_ika-main-A3,rensya_frame-main-A3,saidai_damege-main-A3,range-bullet-main-A3,range-draw-main-A3,damage_min-sub-A3,damage_max-sub-A3,ink-sub-A3,install_num-sub-A3,good-sub-A3,direct_range-sub-A3,distant-range_sub-A3,damage_min-special-A3,damage_max-special-A3,duration-special-A3,good-special-A3,direct_rad-special-A3,close_rad-special-A3,distant_rad-special-A3,category1-A4,category2-A4,subweapon-A4,special-A4,mainweapon-A4,reskin-A4,range-main-A4,atack-main-A4,rapid-main-A4,vote-main-A4,fav-main-A4,DPS-main-A4,kill_time_human-main-A4,kill_time_ika-main-A4,front_gap_human-main-A4,front_gap_ika-main-A4,rensya_frame-main-A4,saidai_damege-main-A4,range-bullet-main-A4,range-draw-main-A4,damage_min-sub-A4,damage_max-sub-A4,ink-sub-A4,install_num-sub-A4,good-sub-A4,direct_range-sub-A4,distant-range_sub-A4,damage_min-special-A4,damage_max-special-A4,duration-special-A4,good-special-A4,direct_rad-special-A4,close_rad-special-A4,distant_rad-special-A4,category1-B1,category2-B1,subweapon-B1,special-B1,mainweapon-B1,reskin-B1,range-main-B1,atack-main-B1,rapid-main-B1,vote-main-B1,fav-main-B1,DPS-main-B1,kill_time_human-main-B1,kill_time_ika-main-B1,front_gap_human-main-B1,front_gap_ika-main-B1,rensya_frame-main-B1,saidai_damege-main-B1,range-bullet-main-B1,range-draw-main-B1,damage_min-sub-B1,damage_max-sub-B1,ink-sub-B1,install_num-sub-B1,good-sub-B1,direct_range-sub-B1,distant-range_sub-B1,damage_min-special-B1,damage_max-special-B1,duration-special-B1,good-special-B1,direct_rad-special-B1,close_rad-special-B1,distant_rad-special-B1,category1-B2,category2-B2,subweapon-B2,special-B2,mainweapon-B2,reskin-B2,range-main-B2,atack-main-B2,rapid-main-B2,vote-main-B2,fav-main-B2,DPS-main-B2,kill_time_human-main-B2,kill_time_ika-main-B2,front_gap_human-main-B2,front_gap_ika-main-B2,rensya_frame-main-B2,saidai_damege-main-B2,range-bullet-main-B2,range-draw-main-B2,damage_min-sub-B2,damage_max-sub-B2,ink-sub-B2,install_num-sub-B2,good-sub-B2,direct_range-sub-B2,distant-range_sub-B2,damage_min-special-B2,damage_max-special-B2,duration-special-B2,good-special-B2,direct_rad-special-B2,close_rad-special-B2,distant_rad-special-B2,category1-B3,category2-B3,subweapon-B3,special-B3,mainweapon-B3,reskin-B3,range-main-B3,atack-main-B3,rapid-main-B3,vote-main-B3,fav-main-B3,DPS-main-B3,kill_time_human-main-B3

## カラムの確認

In [ ]:
print(train_df.columns)

In [ ]:
train_df["period"] = pd.to_datetime(train_df['period'])
test_df["period"] = pd.to_datetime(test_df['period'])

print(train_df["period"].max() == test_df["period"].max())
print(train_df["period"].min() == test_df["period"].min())

In [ ]:
train_df["period"].hist()

In [ ]:
test_df["period"].hist()

In [ ]:
print(train_df["A1-level"].max())
print(train_df["A1-level"].min())
print(train_df["A1-level"].std())

In [ ]:
train_df["category1-A1"].unique()

In [ ]:
train_df["category2-A1"].unique()

In [ ]:
train_df[["category1-A1", "category2-A1"]].head()

In [ ]:
train_df["stage"].unique().shape

In [ ]:
for mode in train_df["mode"].unique():
    
    rate = train_df[train_df["mode"] == mode]["y"].value_counts().loc[1] / train_df[train_df["mode"] == mode].shape[0]
    print("{} : {}".format(mode, rate))

In [ ]:
train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [ ]:
train_df

In [ ]:
col = "disconnection-B"
train_df.groupby(col)["y"].sum() / train_df.groupby(col)["y"].count()

In [ ]:
# rankの欠損値を埋める
train_df, test_df = prepro.fillna_rank(train_df, test_df)

#そのほかの欠損値を埋める
train_df, test_df = prepro.fillna(train_df, test_df)

In [ ]:
train_df, test_df = prepro.level_inverse(train_df, test_df)

In [ ]:
enc.fit(train_df["mode"], train_df["y"])

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
folds = StratifiedKFold(5, shuffle=False)

In [ ]:
for train_fold, valid_fold in folds.split(train_df, train_df["y"].values):
    train_fold_df = train_df.iloc[train_fold]
    enc.fit(train_df["mode"], train_df["y"])
    

In [ ]:
train_df["mode_"] = enc.transform(train_df["mode"], )

In [ ]:
train_df[["mode", "mode_"]][train_df["mode"] == "nawabari"]

In [ ]:
def target_encoder(df1, df2, col, y, n_splits=5):
    kf = KFold(n_splits=n_splits, random_state=random.randint(0, 100000), shuffle=True)
    col_name = col + "-tgtenc"
    df1[col_name] = 0
    for train_index, test_index in kf.split(df1):
        enc = te.TargetEncoder().fit(df1.loc[train_index][col], y[train_index])
        df1[col_name].loc[test_index] = enc.transform(df1.loc[test_index][col])[col]
    enc = te.TargetEncoder().fit(df1[col], y)
    df2[col_name] = enc.transform(df2[col])[col]
    return df1, df2

In [ ]:
import random
train_df, test_df = target_encoder(train_df, test_df, "mode", train_df["y"].values)

In [ ]:
y = train_df["y"]
kf = KFold(n_splits=5, random_state=random.randint(0, 100000), shuffle=True)
col_name = "mode" + "-tgtenc"
train_df[col_name] = 0
for train_index, test_index in kf.split(train_df):
    enc = te.TargetEncoder().fit(train_df.loc[train_index]["mode"], y[train_index])
    train_df[col_name].loc[test_index] = enc.transform(train_df.loc[test_index]["mode"])["mode"]



In [ ]:
train_df